<a href="https://colab.research.google.com/github/fawazshah/Reddit-Analysis/blob/main/5_direct_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

  Using cached https://files.pythonhosted.org/packages/b0/9e/5b80becd952d5f7250eaf8fc64b957077b12ccfe73e9c03d37146ab29712/transformers-4.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a1/88/7b1e45720ecf59c6c6737ff332f41c955963090a18e72acbcbeac6b25e86/huggingface_hub-0.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ae/04/5b870f26a858552025a62f1649c20d29d2672c02ff3c3fb4c688ca46467a/tokenizers-0.10.2-cp37-cp37m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/75/ee/67241dc87f266093c533a2d4d3d69438e57d7a90abb216fa076e7d475d4a/sacremoses-0.0.45-py3-none-any.whl


In [2]:
import numpy as np
import pandas as pd
import random
import requests
from sklearn.metrics import f1_score, accuracy_score, classification_report
import time
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler
import transformers

In [3]:
# Setting random seed and device
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

### BERT setup

In [4]:
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
# Compute the length of the longest sentence in particular column out of
# all train, val and test data
def compute_max_length(df, bert_input_func):

  sentences = bert_input_func(df)

  max_len = 0

  for sent in sentences:

      # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
      input_ids = tokenizer.encode(sent, add_special_tokens=True)

      # Update the maximum sentence length.
      max_len = max(max_len, len(input_ids))

  return max_len

In [7]:
def compute_sentences_article_body(df):
    return list(df['article body'])

def compute_sentences_comment_body(df):
    return list(df['comment body'])

In [8]:
def create_bert_dataset(df, bert_input_func, max_sequence_len):
    # Returns a TensorDataset of sequences extracted from df

    token_ids = []
    token_type_ids = [] # segment ids 
    attention_masks = []

    sentences = bert_input_func(df)

    for sent in sentences:
        encoding_dict = tokenizer(sent,
                                  add_special_tokens=True,
                                  max_length=max_sequence_len,
                                  padding='max_length',
                                  truncation=True,
                                  return_token_type_ids = True,
                                  return_attention_mask = True,
                                  return_tensors = 'pt'
                                  )
        token_ids.append(encoding_dict['input_ids'])
        token_type_ids.append(encoding_dict['token_type_ids'])
        attention_masks.append(encoding_dict['attention_mask'])
    
    token_ids = torch.cat(token_ids, dim=0)
    token_type_ids = torch.cat(token_type_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(df['bias'].values)
    
    return TensorDataset(token_ids, token_type_ids, attention_masks, labels)

In [9]:
def train_BERT(train_dataloader, val_dataloader, model, number_epoch):

    train_loss = []
    valid_loss = []

    optimizer = AdamW(model.parameters(),
                    lr = 2e-5, 
                    eps = 1e-8 
                )

    # Create the learning rate scheduler.
    total_steps = len(train_dataloader) * number_epoch
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, 
                                                num_training_steps=total_steps)

    for epoch in range(1, number_epoch+1):

        # TRAINING

        time0 = time.time()

        model.train()

        epoch_train_loss = 0
        no_observations = 0
        epoch_train_predictions = []
        epoch_train_labels = []

        for batch in train_dataloader:

            # Each batch contains token ids, token type ids, attention masks and labels
            b_token_ids = batch[0].to(device)
            b_token_type_ids = batch[1].to(device)
            b_attention_masks = batch[2].to(device)
            b_labels = batch[3].to(device)

            no_observations = no_observations + b_labels.shape[0]
            
            output = model(b_token_ids, 
                    token_type_ids=b_token_type_ids, 
                    attention_mask=b_attention_masks, 
                    labels=b_labels)

            model.zero_grad()

            loss = output.loss
            logits = output.logits

            predictions = np.argmax(logits.detach().cpu().numpy(), axis=1)
            labels = b_labels.detach().cpu().numpy()
            epoch_train_predictions.extend(predictions)
            epoch_train_labels.extend(labels)

            loss.backward()
            # Clip the norm of the gradients to 1 to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step() 

            # Update the learning rate using the scheduler
            scheduler.step()  

            epoch_train_loss += loss.item()*b_labels.shape[0]

        epoch_train_loss, epoch_train_acc = epoch_train_loss / no_observations, accuracy_score(epoch_train_labels, epoch_train_predictions)

        # VALIDATION

        epoch_valid_loss, epoch_val_predictions, epoch_val_labels = evaluate_BERT(val_dataloader, model)
        epoch_valid_acc = accuracy_score(epoch_val_labels, epoch_val_predictions)

        # FINALLY

        print(f"Epoch took: {time.time() - time0}")

        print(f'| Epoch: {epoch:02} | Train Loss: {epoch_train_loss:.2f} | Train Accuracy: {epoch_train_acc:.2f} | \
        Val. Loss: {epoch_valid_loss:.2f} | Val. Accuracy: {epoch_valid_acc:.2f} |')

        train_loss.append(epoch_train_loss)
        valid_loss.append(epoch_valid_loss)
    
    return train_loss, valid_loss

In [10]:
def evaluate_BERT(test_dataloader, model):

    model.eval()
    total_loss = 0
    no_observations = 0
    predictions_all = []
    labels_all = []

    with torch.no_grad():
        for batch in test_dataloader:
            b_token_ids = batch[0].to(device)
            b_token_type_ids = batch[1].to(device)
            b_attention_masks = batch[2].to(device)
            b_labels = batch[3].to(device)

            no_observations += b_labels.shape[0]
            output = model(b_token_ids, token_type_ids=b_token_type_ids, 
                                        attention_mask=b_attention_masks)
            logits = output.logits
            loss = loss_fn(logits, b_labels)

            predictions = np.argmax(logits.detach().cpu().numpy(), axis=1)
            labels = b_labels.detach().cpu().numpy()
            predictions_all.extend(predictions)
            labels_all.extend(labels)

            total_loss += loss.item()*b_labels.shape[0]
    
    return total_loss / no_observations, predictions_all, labels_all

### Article body -> comments

#### Loading data

In [11]:
submissions_url = 'https://raw.githubusercontent.com/fawazshah/Reddit-Analysis/master/data/collated-data/submissions_preprocessed.tsv'
submissions_df = pd.read_csv(submissions_url, sep='\t')

comments_url = 'https://raw.githubusercontent.com/fawazshah/Reddit-Analysis/master/data/collated-data/comments_preprocessed.tsv'
comments_df = pd.read_csv(comments_url, sep='\t')

In [12]:
print(f"No. submissions: {len(submissions_df)}")
print(f"No. comments: {len(comments_df)}")

No. submissions: 806
No. comments: 11923


#### One hot encode labels

In [13]:
# left == 0
# right == 1

def encode_labels(label):
    if label == "left":
        return 0
    else:
        return 1

submissions_df['bias'] = submissions_df['bias'].apply(encode_labels)
comments_df['bias'] = comments_df['bias'].apply(encode_labels)

#### Split data into train/val/test

In [14]:
# Train/val/test split

TRAIN = 0.7
VAL = 0.1
TEST = 0.2

In [15]:
split_point_1 = int(TRAIN*len(submissions_df))
split_point_2 = int((TRAIN+VAL)*len(submissions_df))

submissions_train_df = submissions_df.iloc[:split_point_1].copy()
submissions_val_df = submissions_df.iloc[split_point_1:split_point_2].copy()
submissions_test_df = submissions_df.iloc[split_point_2:].copy()

In [16]:
print(f"Size of training set: {len(submissions_train_df)}")
print(f"Size of validation set: {len(submissions_val_df)}")
print(f"Size of test set: {len(submissions_test_df)}")

Size of training set: 564
Size of validation set: 80
Size of test set: 162


#### Create BERT dataloaders

In [17]:
dataloaders = {}
BATCH_SIZE = 10

In [18]:
# Using no folds
dataloaders['article bodies'] = {}
train_dataset = create_bert_dataset(submissions_train_df, compute_sentences_article_body, 512)
dataloaders['article bodies']['train'] = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=BATCH_SIZE)
val_dataset = create_bert_dataset(submissions_val_df, compute_sentences_article_body, 512)
dataloaders['article bodies']['val'] = DataLoader(val_dataset, sampler=RandomSampler(val_dataset), batch_size=BATCH_SIZE)
test_dataset = create_bert_dataset(submissions_test_df, compute_sentences_article_body, 512)
dataloaders['article bodies']['test'] = DataLoader(test_dataset, sampler=RandomSampler(test_dataset), batch_size=BATCH_SIZE)

In [19]:
comment_dataset = create_bert_dataset(comments_df, compute_sentences_comment_body, 512)
dataloaders['comment bodies'] = DataLoader(comment_dataset, sampler = RandomSampler(comment_dataset), batch_size=BATCH_SIZE)

#### Run BERT models

In [20]:
NUM_EPOCHS = 5

In [21]:
# Train and test on article bodies

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions = False,
    output_hidden_states = False,
)
model.cuda()

# Train model on article bodies
train_dataloader = dataloaders['article bodies']['train']
val_dataloader = dataloaders['article bodies']['val']
train_loss, valid_loss = train_BERT(train_dataloader, val_dataloader, model, NUM_EPOCHS)

# Test on article bodies
_, predictions, labels = evaluate_BERT(dataloaders['article bodies']['test'], model)

print(classification_report(labels, predictions))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch took: 18.61808180809021
| Epoch: 01 | Train Loss: 0.64 | Train Accuracy: 0.66 |         Val. Loss: 1.33 | Val. Accuracy: 0.14 |
Epoch took: 18.450361490249634
| Epoch: 02 | Train Loss: 0.54 | Train Accuracy: 0.75 |         Val. Loss: 1.13 | Val. Accuracy: 0.36 |
Epoch took: 18.441808462142944
| Epoch: 03 | Train Loss: 0.48 | Train Accuracy: 0.80 |         Val. Loss: 1.17 | Val. Accuracy: 0.41 |
Epoch took: 18.478650331497192
| Epoch: 04 | Train Loss: 0.37 | Train Accuracy: 0.87 |         Val. Loss: 1.27 | Val. Accuracy: 0.42 |
Epoch took: 18.47547960281372
| Epoch: 05 | Train Loss: 0.31 | Train Accuracy: 0.90 |         Val. Loss: 1.49 | Val. Accuracy: 0.40 |
              precision    recall  f1-score   support

           0       0.32      0.56      0.41        45
           1       0.76      0.55      0.64       117

    accuracy                           0.55       162
   macro avg       0.54      0.55      0.52       162
weighted avg       0.64      0.55      0.57       162



In [22]:
# Infer on comments

_, predictions, labels = evaluate_BERT(dataloaders['comment bodies'], model)
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.60      0.12      0.20      5960
           1       0.51      0.92      0.66      5963

    accuracy                           0.52     11923
   macro avg       0.55      0.52      0.43     11923
weighted avg       0.55      0.52      0.43     11923



### Comments -> article body

#### Loading data

In [23]:
submissions_url = 'https://raw.githubusercontent.com/fawazshah/Reddit-Analysis/master/data/collated-data/submissions_preprocessed.tsv'
submissions_df = pd.read_csv(submissions_url, sep='\t')

comments_url = 'https://raw.githubusercontent.com/fawazshah/Reddit-Analysis/master/data/collated-data/comments_preprocessed.tsv'
comments_df = pd.read_csv(comments_url, sep='\t')

In [24]:
print(f"No. submissions: {len(submissions_df)}")
print(f"No. comments: {len(comments_df)}")

No. submissions: 806
No. comments: 11923


#### One hot encode labels

In [25]:
# left == 0
# right == 1

def encode_labels(label):
    if label == "left":
        return 0
    else:
        return 1

submissions_df['bias'] = submissions_df['bias'].apply(encode_labels)
comments_df['bias'] = comments_df['bias'].apply(encode_labels)

#### Split data into train/val/test

In [26]:
# Train/val/test split

TRAIN = 0.7
VAL = 0.1
TEST = 0.2

In [27]:
split_point_1 = int(TRAIN*len(comments_df))
split_point_2 = int((TRAIN+VAL)*len(comments_df))

comments_train_df = comments_df.iloc[:split_point_1].copy()
comments_val_df = comments_df.iloc[split_point_1:split_point_2].copy()
comments_test_df = comments_df.iloc[split_point_2:].copy()

In [28]:
print(f"Size of training set: {len(comments_train_df)}")
print(f"Size of validation set: {len(comments_val_df)}")
print(f"Size of test set: {len(comments_test_df)}")

Size of training set: 8346
Size of validation set: 1192
Size of test set: 2385


#### Create BERT dataloaders

In [29]:
dataloaders = {}
BATCH_SIZE = 10

In [30]:
# Using no folds
dataloaders['comment bodies'] = {}
train_dataset = create_bert_dataset(comments_train_df, compute_sentences_comment_body, 512)
dataloaders['comment bodies']['train'] = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=BATCH_SIZE)
val_dataset = create_bert_dataset(comments_val_df, compute_sentences_comment_body, 512)
dataloaders['comment bodies']['val'] = DataLoader(val_dataset, sampler=RandomSampler(val_dataset), batch_size=BATCH_SIZE)
test_dataset = create_bert_dataset(comments_test_df, compute_sentences_comment_body, 512)
dataloaders['comment bodies']['test'] = DataLoader(test_dataset, sampler=RandomSampler(test_dataset), batch_size=BATCH_SIZE)

In [31]:
article_dataset = create_bert_dataset(submissions_df, compute_sentences_article_body, 512)
dataloaders['article bodies'] = DataLoader(article_dataset, sampler = RandomSampler(article_dataset), batch_size=BATCH_SIZE)

#### Run BERT models

In [32]:
NUM_EPOCHS = 5

In [33]:
# Train and test on comment bodies

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions = False,
    output_hidden_states = False,
)
model.cuda()

# Train model on comment bodies
train_dataloader = dataloaders['comment bodies']['train']
val_dataloader = dataloaders['comment bodies']['val']
train_loss, valid_loss = train_BERT(train_dataloader, val_dataloader, model, NUM_EPOCHS)

# Test on comment bodies
_, predictions, labels = evaluate_BERT(dataloaders['comment bodies']['test'], model)

print(classification_report(labels, predictions))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch took: 272.2363386154175
| Epoch: 01 | Train Loss: 0.57 | Train Accuracy: 0.72 |         Val. Loss: 1.42 | Val. Accuracy: 0.11 |
Epoch took: 272.4717540740967
| Epoch: 02 | Train Loss: 0.48 | Train Accuracy: 0.78 |         Val. Loss: 1.06 | Val. Accuracy: 0.41 |
Epoch took: 272.3404908180237
| Epoch: 03 | Train Loss: 0.34 | Train Accuracy: 0.86 |         Val. Loss: 1.91 | Val. Accuracy: 0.34 |
Epoch took: 272.5943124294281
| Epoch: 04 | Train Loss: 0.22 | Train Accuracy: 0.92 |         Val. Loss: 2.38 | Val. Accuracy: 0.40 |
Epoch took: 272.5154809951782
| Epoch: 05 | Train Loss: 0.16 | Train Accuracy: 0.95 |         Val. Loss: 2.62 | Val. Accuracy: 0.43 |
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.42      0.59      2385

    accuracy                           0.42      2385
   macro avg       0.50      0.21      0.30      2385
weighted avg       1.00      0.42      0.59      2385



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
# Infer on articles

_, predictions, labels = evaluate_BERT(dataloaders['article bodies'], model)
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.68      0.75      0.72       419
           1       0.70      0.62      0.66       387

    accuracy                           0.69       806
   macro avg       0.69      0.69      0.69       806
weighted avg       0.69      0.69      0.69       806

